In [12]:
import pandas as pd
import numpy as np

In [24]:
LOW = 0.08
HIGH = 0.85

In [25]:
#load prediction file for both detection and classification
det = pd.read_csv('../checkpoint/post_comp/vfnet/submission/fold1.csv')
cls = pd.read_csv('../checkpoint/1703/classifier/default.csv')

In [26]:
print(det.shape)
det.head()

(3000, 2)


,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,0 0.31 1144 640 1437 930 0 0.13 981 638 1436 1...
1,004f33259ee4aef671c2b95d54e4be68,0 0.81 1267 582 1532 889 0 0.15 1066 580 1537 ...
2,008bdde2af2462e86fd373a445d0f4cd,0 0.85 1427 819 1728 1188 0 0.33 1203 816 1728...
3,009bc039326338823ca3aa84381f17f1,0 0.60 992 461 1217 721 0 0.17 785 464 1222 85...
4,00a2145de1886cb9eb88869c85d74080,0 0.69 1116 710 1354 937 0 0.15 1181 708 1353 ...


In [27]:
print(cls.shape)
cls.head()

(3000, 2)


,image_id,target
0,002a34c58c5b758217ed1f584ccbcfe9,0.013326
1,004f33259ee4aef671c2b95d54e4be68,0.037235
2,008bdde2af2462e86fd373a445d0f4cd,0.939700
3,009bc039326338823ca3aa84381f17f1,0.123799
4,00a2145de1886cb9eb88869c85d74080,0.654006


In [28]:
final_pred = pd.merge(det, cls, on = 'image_id', how = 'left')
print(final_pred.shape)
final_pred.head()

(3000, 3)


,image_id,PredictionString,target
0,002a34c58c5b758217ed1f584ccbcfe9,0 0.31 1144 640 1437 930 0 0.13 981 638 1436 1...,0.013326
1,004f33259ee4aef671c2b95d54e4be68,0 0.81 1267 582 1532 889 0 0.15 1066 580 1537 ...,0.037235
2,008bdde2af2462e86fd373a445d0f4cd,0 0.85 1427 819 1728 1188 0 0.33 1203 816 1728...,0.939700
3,009bc039326338823ca3aa84381f17f1,0 0.60 992 461 1217 721 0 0.17 785 464 1222 85...,0.123799
4,00a2145de1886cb9eb88869c85d74080,0 0.69 1116 710 1354 937 0 0.15 1181 708 1353 ...,0.654006


In [29]:
#Number of 'No Findings' before filtering
final_pred['PredictionString'].value_counts().iloc[[0]]

0 0.67 1620 945 1913 1252 0 0.29 1248 945 1916 1558 0 0.29 1494 940 1916 1348 0 0.25 1696 948 1914 1190 0 0.14 1546 944 1926 1544 0 0.13 1269 946 1912 1298 0 0.09 1261 931 1907 1953 0 0.09 1776 975 1912 1146 0 0.08 1661 937 1908 1117 0 0.08 1762 957 1917 1240 0 0.07 1247 920 1673 1886 0 0.06 1453 917 1929 1799 0 0.06 1481 912 1906 1204 2 0.07 1730 949 1910 1128 2 0.06 1656 945 1914 1189 2 0.06 1776 975 1912 1146 3 0.17 1192 1714 2239 2066 3 0.10 1619 945 1913 1247 5 0.10 2089 1333 2493 1921 5 0.08 1952 877 2519 1951 5 0.07 669 1205 1184 1873 5 0.06 827 1224 1027 1393 5 0.05 1813 519 2586 1988 5 0.05 2134 1089 2552 1919 7 0.10 563 1805 1228 2126 7 0.09 1693 505 1993 737 7 0.08 1690 500 2142 702 7 0.07 1238 1029 1507 1754 7 0.07 1713 1235 1928 1448 7 0.06 1680 521 1955 826 7 0.06 909 1762 1071 1842 7 0.05 544 1825 1316 2252 8 0.08 1984 1297 2052 1356 8 0.08 1713 1235 1928 1448 8 0.06 920 1764 1062 1840 8 0.06 1800 1168 1909 1262 8 0.06 747 1460 798 1520 9 0.21 560 1808 1256 2140 9 0.12 1

In [30]:
def filter_class(row, low_thr=LOW, high_thr=HIGH):
    prob = row['target']
    if prob<low_thr:
        ## Less chance of having any disease
        row['PredictionString'] = '14 1 0 0 1 1'
    elif low_thr<=prob<high_thr:
        ## More chance of having any diesease
        row['PredictionString']+=f' 14 {prob} 0 0 1 1'
    elif high_thr<=prob:
        ## Good chance of having any disease so believe in object detection model
        row['PredictionString'] = row['PredictionString']
    else:
        raise ValueError('Prediction must be from [0-1]')
    return row

In [31]:
submission = final_pred.apply(filter_class, axis=1)
submission.head()

,image_id,PredictionString,target
0,002a34c58c5b758217ed1f584ccbcfe9,14 1 0 0 1 1,0.013326
1,004f33259ee4aef671c2b95d54e4be68,14 1 0 0 1 1,0.037235
2,008bdde2af2462e86fd373a445d0f4cd,0 0.85 1427 819 1728 1188 0 0.33 1203 816 1728...,0.939700
3,009bc039326338823ca3aa84381f17f1,0 0.60 992 461 1217 721 0 0.17 785 464 1222 85...,0.123799
4,00a2145de1886cb9eb88869c85d74080,0 0.69 1116 710 1354 937 0 0.15 1181 708 1353 ...,0.654006


In [32]:
#Number of 'No Findings' after filtering
submission['PredictionString'].value_counts().iloc[[0]]

14 1 0 0 1 1    1912
Name: PredictionString, dtype: int64

In [33]:
submission[['image_id', 'PredictionString']].to_csv('../checkpoint/post_comp/vfnet/submission/submission.csv',index = False)